<a href="https://colab.research.google.com/github/Narendrakumar-Suresh/CNN_MNIST/blob/main/Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_of_epochs=20
batch_size=64
learning_rate=0.0005

In [36]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))])

train_data=torchvision.datasets.FashionMNIST(root='./data',train=True,download=True,transform=transform)
test_data=torchvision.datasets.FashionMNIST(root='./data',train=False,download=True,transform=transform)

train_loader=torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_loder=torch.utils.data.DataLoader(test_data, batch_size=batch_size,shuffle=False)

In [37]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()

        self.conv1=nn.Conv2d(1,32,3)
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(32,16,3)

        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,84)
        self.fc4=nn.Linear(84,10)

    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=x.view(x.size(0),-1)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=F.relu(self.fc3(x))
        x=self.fc4(x)

        return x

In [38]:
model=CNN().to(device)

criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

n_steps=len(train_loader)

In [32]:
print('Training started')

for epoch in range(num_of_epochs):
    for i, (sample,label) in enumerate(train_loader):
        sample=sample.to(device)
        label=label.to(device)

        output=model(sample)
        loss=criterion(output,label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'epoch{epoch+1}/{num_of_epochs}: loss: {loss.item():.4f}')

print('Training Completed!!!')

Training started
epoch1/20: loss: 0.7430
epoch2/20: loss: 0.3881
epoch3/20: loss: 0.5240
epoch4/20: loss: 0.4072
epoch5/20: loss: 0.3557
epoch6/20: loss: 0.3659
epoch7/20: loss: 0.3399
epoch8/20: loss: 0.2712
epoch9/20: loss: 0.1721
epoch10/20: loss: 0.2334
epoch11/20: loss: 0.1455
epoch12/20: loss: 0.2676
epoch13/20: loss: 0.1855
epoch14/20: loss: 0.0666
epoch15/20: loss: 0.2465
epoch16/20: loss: 0.2491
epoch17/20: loss: 0.0698
epoch18/20: loss: 0.2653
epoch19/20: loss: 0.2018
epoch20/20: loss: 0.0795
Training Completed!!!


In [33]:
with torch.no_grad():
    n_correct=0
    n_samples=0

    n_correct_classes=[0 for _ in range(10)]
    n_correct_samples=[0 for _ in range(10)]

    for sample,label in test_loder:
        sample=sample.to(device)
        label=label.to(device)

        output=model(sample)

        prob=F.softmax(output,dim=1)
        _,predicted=torch.max(prob,1)

        n_samples += label.size(0)
        n_correct += (predicted == label).sum().item()

        for i in range(len(label)):
            lab=label[i].item()
            pred=predicted[i].item()

            if lab == pred:
                n_correct_classes[lab] += 1
            n_correct_samples[lab] += 1

    acc=100*(n_correct/n_samples)
    print(f'Accuracy is {acc}%')


Accuracy is 90.75999999999999%
